In [8]:
import json
from PIL import Image
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

### preprocess

In [44]:
visual_news_dir = "/data3/qianqi/dataset/visual_news/origin"
news_clippings_dir = "/data3/qianqi/dataset/news_clippings/data/merged_balanced"
evidence_dir = "/data3/qianqi/dataset/queries_dataset"

In [45]:
visual_news_data = json.load(open(f"{visual_news_dir}/data.json"))
visual_news_data_mapping = {ann["id"]: ann for ann in visual_news_data}
print(len(visual_news_data_mapping))
for k, v in visual_news_data_mapping[39136].items(): 
    print(k, v)
# img = Image.open(f"{visual_news_dir}/{visual_news_data_mapping[39136]['image_path']}")
# plt.imshow(img)
# plt.axis("off")
# plt.show()

1259732
id 39136
caption Candace Pickens and her son Zachaeus
topic law_crime
source washington_post
image_path ./washington_post/images/0376/501.jpg
article_path ./washington_post/articles/39136.txt


In [ ]:
news_clippings_data = json.load(open(f"{news_clippings_dir}/train.json"))
news_clippings_data = news_clippings_data["annotations"]
print(len(news_clippings_data))
print(news_clippings_data[0])
ann = news_clippings_data[0]

caption = visual_news_data_mapping[ann["id"]]["caption"]
image_path = visual_news_data_mapping[ann["image_id"]]["image_path"]

print("Caption: ", caption)
print("Image Path: ", image_path)
print("Is Falsified: ", ann["falsified"])

71072
{'id': 620701, 'image_id': 620701, 'similarity_score': 1, 'source_dataset': 2, 'falsified': False}
Caption:  Saudi troops cheer as they ride at the back of an army truck in the southwestern province of Jizan near the border with Yemen
Image Path:  ./guardian/images/0338/482.jpg
Is Falsified:  False


In [20]:
evidence_data = json.load(open(f"{evidence_dir}/dataset_items_train.json"))

In [21]:
print(len(evidence_data))
top_k = 10

evidence_captions = {}

for key, data in tqdm(evidence_data.items()):
    inv_dir = os.path.join(evidence_dir, "merged_balanced", data["inv_path"])
    # inverse image search
    try:
        captions = json.load(open(f"{inv_dir}/captions_info"))
        cap_idx = json.load(open(f"{inv_dir}/captions_to_keep_idx"))
        captions = [captions["captions"][idx] for idx in list(cap_idx.values())[0]][:top_k]
        evidence_captions[int(key)] = captions
    except:
        # no searched results available, do nothing
        continue

70561


100%|██████████| 70561/70561 [00:03<00:00, 23248.02it/s]


In [28]:
# 53k training data with evidence
training_data = []
for idx in evidence_captions:
    ann = news_clippings_data[idx]
    caption = visual_news_data_mapping[ann["id"]]["caption"]
    image_path = visual_news_data_mapping[ann["image_id"]]["image_path"]
    label = ann["falsified"]
    evidence = evidence_captions[idx]
    # print(caption)
    # print(image_path)
    # img = Image.open(f"{visual_news_dir}/{image_path}")
    # plt.imshow(img)
    # plt.axis("off")
    # plt.show()
    # print(evidence)
    # print(f"Is fake: {label}")
    training_data.append({
        "image" : image_path,
        "caption" : caption,
        "fake" : label,
        "evidence" : evidence
    })

In [32]:
print(len(training_data))
print(training_data[0])

print(f"{sum([1 if d['fake'] else 0 for d in training_data]) / len(training_data)} fake")

53316
{'image': './guardian/images/0338/482.jpg', 'caption': 'Saudi troops cheer as they ride at the back of an army truck in the southwestern province of Jizan near the border with Yemen', 'fake': False, 'evidence': ['Inside Mazrak refugee camp, Yemen | World news | The Guardian', 'Mazrak camp in Yemen : Saudi troops cheer as they ride at the b', '/tg/ - Traditional Games', '45 KB', '&quot;friday-lunch-club&quot;: Inside Mazrak refugee camp, Yemen', 'Mazrak camp in Yemen : Saudi troops cheer as they ride at the b', 'Askeriye Platformu | TaleWorlds Forums', '9z30e.jpg', 'Saudi Army Photos and Premium High Res Pictures - Getty Images', 'Southwestern Jizan Stock-Fotos und Bilder - Getty Images']}
0.4981056343311576 fake


### grpo

In [10]:
visual_news_dir = "/data3/qianqi/dataset/visual_news/origin"
news_clippings_dir = "/data3/qianqi/dataset/news_clippings/data/merged_balanced"
evidence_dir = "/data3/qianqi/dataset/queries_dataset"

def get_data(split, top_k_evidence=10):
    visual_news_data = json.load(open(f"{visual_news_dir}/data.json"))
    visual_news_data_mapping = {ann["id"]: ann for ann in visual_news_data}

    news_clippings_data = json.load(open(f"{news_clippings_dir}/{split}.json"))
    news_clippings_data = news_clippings_data["annotations"]

    evidence_data = json.load(open(f"{evidence_dir}/dataset_items_{split}.json"))
    evidence_captions = {}
    for key, data in tqdm(evidence_data.items()):
        inv_dir = os.path.join(evidence_dir, "merged_balanced", data["inv_path"])
        # inverse image search
        try:
            captions = json.load(open(f"{inv_dir}/captions_info"))
            cap_idx = json.load(open(f"{inv_dir}/captions_to_keep_idx"))
            captions = [captions["captions"][idx] for idx in list(cap_idx.values())[0]][:top_k_evidence]
            evidence_captions[int(key)] = captions
        except:
            # no searched results available, do nothing
            continue
    
    data = []
    for idx in tqdm(evidence_captions):
        ann = news_clippings_data[idx]
        caption = visual_news_data_mapping[ann["id"]]["caption"]
        image_path = visual_news_data_mapping[ann["image_id"]]["image_path"]
        if ann["falsified"]: gt_ans = "No"
        else: gt_ans = "Yes"

        evidence = evidence_captions[idx]
        evidence_str = ""
        for i, e in enumerate(evidence): evidence_str += f"Evidence {i + 1}: {e}\n"
        evidence_str = evidence_str.strip()

        prompt = f'''Please give the authenticity of the given image-claim from <'Yes', 'No'>.

Claim: 
{caption}'''

#         prompt = f'''Please give the authenticity of the given image-claim from <'Yes', 'No'>. Additional textual evidence collected from reverse-image search results is provided as well. They may be noisy or partly inaccurate, but often provide helpful context. 

# Claim: 
# {caption}


# Additional textual evidence: 
# {evidence_str}'''

        data.append({
            "image" : image_path,
            "problem" : prompt,
            "gt_ans" : gt_ans
        })
    return data

In [11]:
test_without_evidence = get_data('test')
with open("test_no_evidence.jsonl", 'w') as f:
    for d in test_without_evidence: f.write(json.dumps(d) + "\n")

100%|██████████| 5208/5208 [00:00<00:00, 165364.09it/s]


In [66]:
training_data = get_data("train")
print(f"{sum([1 if d['gt_ans']== 'Yes' else 0 for d in training_data]) / len(training_data)} true")
validation_data = get_data("val")
print(f"{sum([1 if d['gt_ans']== 'Yes' else 0 for d in validation_data]) / len(validation_data)} true")
test_data = get_data("test")
print(f"{sum([1 if d['gt_ans']== 'Yes' else 0 for d in test_data]) / len(test_data)} true")

100%|██████████| 53316/53316 [00:00<00:00, 201153.28it/s]


0.5018943656688424 true


100%|██████████| 5047/5047 [00:00<00:00, 175073.42it/s]


0.5028729938577373 true


100%|██████████| 5208/5208 [00:00<00:00, 174915.20it/s]


0.4988479262672811 true


In [67]:
with open("train.jsonl", 'w') as f:
    for d in training_data: f.write(json.dumps(d) + "\n")
with open("val.jsonl", 'w') as f:
    for d in validation_data: f.write(json.dumps(d) + "\n")
with open("test.jsonl", 'w') as f:
    for d in test_data: f.write(json.dumps(d) + "\n")